In [1]:
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep

In [2]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    method = method.upper()
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: # 실제 수집할 때, 제약사항이 많으므로 여기선 잠시 해제
    try:
        resp = request(method, url,
               params=params if method=='GET' else {},
               data=params if method=='POST' else {},
               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(1) # => random
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp

In [209]:
import sqlite3

conn = sqlite3.connect('koreapas.db')
cur = conn.cursor()

In [230]:
cur.executescript('''
    DROP TABLE IF EXISTS clubs;
    CREATE TABLE clubs(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        title TEXT NOT NULL,
        link TEXT NOT NULL,
        image TEXT,
        description TEXT
    );
''')

In [231]:
url = 'https://www.koreapas.com/bbs/zboard.php'
headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
}

In [232]:
for i in range(1,21):
    print(str(i)+'th page')
    params={
        'id': 'circle',
        'page': i,
        'select_arrange': 'headnum',
        'desc': 'asc',
        'category': '',
        'sn': 'off',
        'ss': 'on',
        'sc': 'on',
        'tagkeyword': '',
        'keyword': '',
        'sn1': '',
        'divpage': 3
    }
    resp = download(url, params = params, headers = headers, method='GET')
    dom = BeautifulSoup(resp.content.decode('euc-kr'),'lxml')

    for _ in dom.select('#revolution_main_table tr[class&=list]'):
        td = _.find_all('td')
        a = _.find_all('a')
        #title
        title = td[2].text.strip()[1:-1]
        title = title.replace('\n','')
        newurl = urljoin(url,re.search(r'href="([^"]+)"',a[0].decode().replace('amp;','')).group(1))
        #link
        link = newurl
        nurl = urllib.request.urlopen(newurl)
        ndom = BeautifulSoup(nurl, "html.parser")
        #image
        image = str([_['src'] for _ in ndom.find_all('img', {'class':'imgur adsbygoogle'})])
        #description
        description = [_.text.strip() for _ in ndom.select('#bonmoon > div > tr > td > div')][0]
        cur.execute('INSERT INTO clubs VALUES(NULL,?,?,?,?)',[title,link,image,description])
        conn.commit()

1th page
2th page
3th page
4th page
5th page
6th page
7th page
8th page
9th page
10th page
11th page
12th page
13th page
14th page
15th page
16th page
17th page
18th page
19th page
20th page


In [259]:
import pandas as pd
pd.read_csv('clubs.csv')

,id,title,link,image,description
0,1,서울권 운동(스포츠) 연합 커뮤니티 STREET BOND,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/d5e4e89/resize'],"📌 STREET BOND 소개\r\n운동은 하고 싶은데, 몇 달 회비 끊고 하자니,..."
1,2,안녕하세요 한일문화연구회에서 함께할 동아리원을 모집합니다,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/d3e4ae1/resize'],일본문화에 관심이 있으신가요? 심심한데 마땅한 동아리가 없으신가요? 중앙동아리 한일...
2,3,2020학년도 2학기 KUISA 14기 벗 모집이 하루 남았습니다!!!!!!,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/2b9355e/resize'],"👉2020년도 2학기 KUISA 14기 벗 모집 안내👈\n\r\nKUISA란, Ko..."
3,4,1분 명상법~! ,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/484c9e2/resize', ...",안녕하세요! 원불교 학생회입니다. ㅎㅎ\r\n이번에는 명상을 어떻게 하는 지에 대한...
4,5,연합발표토론동아리 POP 22기 모집합니다 !,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/607084d/resize', ...",[22기 P.O.P 단원 모집!]\n\r\n📢 스펙업 대표동아리 Top30\r\n📢...
5,6,🔥D-2!!!! 쿠이사 2020-2 벗 모집🔥,https://www.koreapas.com/bbs/view.php?id=circl...,"['https://img.koreapas.com/i/1606ae5/resize', ...","👉2020년도 2학기 KUISA 14기 벗 모집 안내👈\n\r\nKUISA란, Ko..."
6,7,⭐경영대학 글로벌 홍보대사 쿱스버디 21기를 모집합니다,https://www.koreapas.com/bbs/view.php?id=circl...,[],🧒🏻경영대학 글로벌 홍보대사 KUBS BUDDY 21기 모집👦🏻\n\r\n무려 33...
7,8,2020-2학기 KUISA 활동을 함께할 벗을 모집합니다 :),https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/ca9c706/resize'],"👉2020년도 2학기 KUISA 14기 벗 모집 안내👈\n\r\nKUISA란, Ko..."
8,9,[KU_BIG] 고려대학교 빅데이터 연구회 KU-BIG에서 12기 신입부원을 모집합...,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/eb589c3/resize'],[KU_BIG] 고려대학교 빅데이터 연구회 KU-BIG에서 12기 신입부원을 모집합...
9,10,👉2020년도 2학기 KUISA 14기를 함께할 벗들을 모집합니다 :) 👈 (07/...,https://www.koreapas.com/bbs/view.php?id=circl...,['https://img.koreapas.com/i/8843ab5/resize'],"👉2020년도 2학기 KUISA 14기 벗 모집 안내👈\n\r\nKUISA란, Ko..."
